In [4]:
import pandas as pd
import sqlite3
db_file = 'sqlite_db_pythonsqlite.db'
conn = sqlite3.connect(db_file)



* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. *

In [4]:
query1 = """
SELECT name FROM Facilities
WHERE membercost > 0.0;
"""
pd.read_sql_query(query1, conn)

,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


* Q2: How many facilities do not charge a fee to members? *

In [6]:
query2 = """
SELECT COUNT(name) FROM `Facilities` WHERE membercost!= 0"""
pd.read_sql_query(query2, conn)

,COUNT(name)
0,5


* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. 

In [7]:
query3= """SELECT facid, name, membercost, monthlymaintenance 
FROM Facilities 
WHERE membercost > 0 
AND membercost < 0.2 * (monthlymaintenance);
 """

pd.read_sql_query(query3, conn)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. 

In [8]:
query4= """ SELECT * FROM Facilities WHERE facid IN (1,5)"""
pd.read_sql_query(query4, conn)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


 * Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. 

In [ ]:
query5= """SELECT facid, name, membercost, monthlymaintenance,
CASE 
WHEN monthlymaintenance > 100 THEN 'Expensive'
ELSE 'cheap' 
END AS Price
FROM Facilities""" 
pd.read_sql_query(query5, conn)

* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

In [14]:
query6= """SELECT * FROM `Members` 
WHERE `joindate` = (SELECT MAX(`joindate`) FROM Members);
"""
pd.read_sql_query(query6, conn)

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,37,Smith,Darren,"3 Funktown, Denzington, Boston",66796,(822) 577-3541,,2012-09-26 18:08:45


* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [11]:
query7= """SELECT m.surname, m.firstname, f.name
FROM Members AS m
INNER JOIN Bookings as b 
ON m.memid = b.memid
INNER JOIN Facilities as f
ON b.facid = f.facid
WHERE name LIKE '%Tennis Court%'
ORDER by m.surname
 """
pd.read_sql_query(query7, conn)

,surname,firstname,name
0,Bader,Florence,Tennis Court 2
1,Bader,Florence,Tennis Court 2
2,Bader,Florence,Tennis Court 2
3,Bader,Florence,Tennis Court 2
4,Bader,Florence,Tennis Court 2
...,...,...,...
792,Tracy,Burton,Tennis Court 1
793,Tracy,Burton,Tennis Court 1
794,Tracy,Burton,Tennis Court 1
795,Tracy,Burton,Tennis Court 1


* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [12]:
query8= """SELECT firstname || ' ' || surname AS member, f.guestcost
From Members AS m
INNER JOIN Bookings AS b
ON m.memid = b.memid
INNER JOIN Facilities AS f
ON f.facid = f.facid
Where b.starttime LIKE '%2012-09-14%' and f.guestcost > 30
ORDER BY starttime DESC
 """
pd.read_sql_query(query8, conn)

,member,guestcost
0,Tracy Smith,80
1,Tracy Smith,80
2,GUEST GUEST,80
3,GUEST GUEST,80
4,Matthew Genting,80
...,...,...
119,Darren Smith,80
120,Jack Smith,80
121,Jack Smith,80
122,Tim Rownam,80


* Q9: This time, produce the same result as in Q8, but using a subquery.

In [13]:
query9= """SELECT
firstname || ' ' || surname AS member,
name AS Facility,
cost
FROM(SELECT firstname, surname, name,
CASE WHEN firstname = 'GUEST' THEN guestcost 
ELSE membercost END AS cost, starttime
FROM Members
INNER JOIN Bookings
ON Members.memid = Bookings.memid
INNER JOIN Facilities
ON Bookings.facid = Facilities.facid) AS inner_table
WHERE starttime >= '2012-09-14' AND starttime < '2012-09-15'
And cost > 30
ORDER BY cost DESC;
"""
pd.read_sql_query(query9, conn)

,member,Facility,cost
0,GUEST GUEST,Massage Room 1,80
1,GUEST GUEST,Massage Room 1,80
2,GUEST GUEST,Massage Room 1,80
3,GUEST GUEST,Massage Room 2,80


* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [15]:
query10 = """SELECT * 
        FROM (
            SELECT sub.facility, SUM( sub.cost ) AS total_revenue
            FROM (
                SELECT Facilities.name AS facility, 
                    CASE WHEN Bookings.memid =0
                    THEN Facilities.guestcost * Bookings.slots
                    ELSE Facilities.membercost * Bookings.slots
                    END AS cost
                FROM Bookings
                
                INNER JOIN Facilities ON Bookings.facid = Facilities.facid
                INNER JOIN Members ON Bookings.memid = Members.memid
                )sub
            GROUP BY sub.facility
        )sub2
        WHERE sub2.total_revenue <1000; """
pd.read_sql_query(query10, conn)

,facility,total_revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [16]:
query11 = """SELECT a.firstname, a.surname, b.firstname, b.surname
FROM Members a
INNER JOIN Members b
ON a.memid = b.recommendedby
ORDER BY b.surname, b.firstname"""
pd.read_sql_query(query11, conn)

,firstname,surname,firstname,surname
0,Ponder,Stibbons,Florence,Bader
1,Ponder,Stibbons,Anne,Baker
2,Jemima,Farrell,Timothy,Baker
3,Tim,Rownam,Tim,Boothe
4,Darren,Smith,Gerald,Butters
5,Timothy,Baker,Joan,Coplin
6,Tracy,Smith,Erica,Crumpet
7,Janice,Joplette,Nancy,Dare
8,Gerald,Butters,Matthew,Genting
9,Millicent,Purview,John,Hunt


* Q12: Find the facilities with their usage by member, but not guests

In [21]:
query12 = """SELECT f.name, COUNT(b.memid)
FROM Facilities as f
INNER JOIN Bookings as b
ON f.facid = b.facid
WHERE memid > 0
GROUP BY f.name"""
pd.read_sql_query(query12, conn)

,name,COUNT(b.memid)
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


* Q13: Find the facilities usage by month, but not guests

In [ ]:

query13 ="""SELECT b.month, COUNT(b.memid)
FROM (SELECT EXTRACT(MONTH FROM starttime) AS month, memid
        FROM Bookings
        WHERE memid !=0) AS b
        GROUP BY b.month"""
pd.read_sql_query(query13, conn)